In [1]:
import os
import json
import sqlite3

from typing import List, Dict, Any, Tuple, Optional

from formating import convert_json_to_insert_sql, convert_json_to_update_sql
from crud import insert_records, delete_records, fetch_records, get_tables

In [2]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
assets_dir = os.path.join(parent_dir, '_assets')
db_dir = os.path.join(parent_dir, 'fetusapp')
db_filename = 'data.sqlite'
data_filename = 'detoraki_fetusdata.json'
mapping_filename = 'column_mapping.json'

db_path = os.path.join(db_dir, db_filename)
json_file = os.path.join(assets_dir, data_filename)
mapping_file = os.path.join(current_dir, mapping_filename)

Load the source data

In [3]:
with open(json_file, 'r', encoding='utf-8') as f:
        json_data = json.load(f)

Load the column mapping from the JSON file

In [4]:
with open(mapping_file, 'r', encoding='utf-8') as f:
        column_mapping = json.load(f)

### Setup admin in Users (INSERT)

The script should already be there in _assets

In [ ]:
target_tablename = 'users'
insert_records(target_tablename, db_path)


In [ ]:
fetch_records(target_tablename, db_path)

In [ ]:
# delete_records(target_tablename, db_path)

---------------------

### Migrate Patients (INSERT)

Includes manual step to treat patients missing last_name

In [ ]:
source_tablename = 's_org_ext'
target_tablename = 'patients'
data_index = 3

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

#### Data Cleansing:
* Identify records with null last name and either 
    * convert the null to the string 'null' in the generated script or
    * remove the record altogether if it doesn't have any useful payload

In [ ]:
patient_data = json_data[data_index]['data']
null_lastname_ids = [item['s_org_extID'] for item in patient_data if item['lastName'] == ""]
print(null_lastname_ids)

In [ ]:
delete_records(target_tablename, db_path)

In [ ]:
insert_records(target_tablename, db_path)

In [ ]:
fetch_records(target_tablename, db_path)

---------------------

### Migrate MedicalHistory (INSERT)

In [ ]:
source_tablename = 'tblmedhistory'
target_tablename = 'history_medical'
data_index = 10

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
insert_records(target_tablename, db_path)

In [ ]:
# delete_records(target_tablename, db_path)

In [ ]:
fetch_records(target_tablename, db_path)

------------------

### Migrate Short Gynaecological History data (UPDATE)

In [ ]:
source_tablename = 'tblgynhistoryshort'
target_tablename = 'history_medical'
data_index = 7

sql_output = convert_json_to_update_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename, source_id = 'MedHistoryID', target_id = 'id')
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
insert_records(target_tablename, db_path)

---------

### Migrate Pathologies (UPDATE)

In [ ]:
source_tablename = 'tblpathologies'
target_tablename = 'history_medical'
data_index = 13

sql_output = convert_json_to_update_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename, source_id = 'MedHistoryID', target_id = 'id')
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
insert_records(target_tablename, db_path)

In [ ]:
fetch_records(target_tablename, db_path)

----------

### Migrate ObstetricsHistory (INSERT)

In [ ]:
source_tablename = 'tblobstetrichistory'
target_tablename = 'history_obstetrics'
data_index = 11 # this is the index in the json_data list where the relevant data is stored

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
delete_records(target_tablename, db_path)

In [ ]:
insert_records(target_tablename, db_path)

In [ ]:
fetch_records(target_tablename, db_path)

### tblobstetrichistory_x NOT TO BE MIGRATED

### Migrate Short Pregnancies History data (INSERT)

In [ ]:
source_tablename = 'tblpregnancyhistory'
target_tablename = 'pregnancy_history'
data_index = 16 # this is the index in the json_data list where the relevant data is stored

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
delete_records(target_tablename, db_path)

In [ ]:
insert_records(target_tablename, db_path)

In [ ]:
fetch_records(target_tablename, db_path)

### Migrate Short Pregnancies History X data (INSERT)

In [ ]:
source_tablename = 'tblpregnancyhistory_x'
target_tablename = 'pregnancy_history_x'
data_index = 17 # this is the index in the json_data list where the relevant data is stored

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
insert_records(target_tablename, db_path)
fetch_records(target_tablename, db_path)
# delete_records(target_tablename, db_path)

Delete records with null TER

In [ ]:
import sqlite3

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("""
DELETE FROM pregnancy_history_x
WHERE pregnancy_id IN (SELECT id FROM pregnancy_history WHERE ter IS NULL)
""")
cursor.execute("""
DELETE FROM pregnancy_history
WHERE ter IS NULL
""")
conn.commit()

conn.close()

### Migrate Gyn History X data (INSERT)

DELETE 5,8 and 10 records

In [5]:
source_tablename = 'tblgynhistory'
target_tablename = 'gyn_history'
data_index = 6 # this is the index in the json_data list where the relevant data is stored

sql_output = convert_json_to_insert_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [ ]:
insert_records(target_tablename, db_path)
fetch_records(target_tablename, db_path)
# delete_records(target_tablename, db_path)

(1, 3, 1, 1, '2014-01-23 02:00:29', '2014-02-21 09:55:54', '2014-01-22', 'Πρώτο Ραντεβού', '2012-02-07', 'Όχι κάτι σοβαρό', 'Τι να λέμε', 36, 12, 5, 'καλή δείχνει', 'μακαρόνια με σάλτσα', 164, 50, 'καλά', 'καθαροί', 'επίπεδη', 'οκ', 'οκ', 'οκ', 'οκ', 'οκ', 'Ok', 1)
(2, 3, 1, 1, '2014-01-23 02:02:47', '2014-01-23 02:03:32', '2014-01-01', 'ναι', '2012-03-08', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 0)
(3, 3, 1, 1, '2014-01-23 02:03:38', '2014-01-23 13:35:53', '2014-01-09', 'Να δω το παιδάκι μου', '2014-01-09', 'Mobile', 'Διακοπές το καλοκαίρι', 21, 10, 55, 'σε γνωρίζω από την...', 'και από αυτό δεν πάμε πίσω', 165, 50, None, None, None, None, None, None, None, None, None, 1)
(4, 3, 1, 1, '2014-01-28 21:42:48', '2014-01-28 21:43:21', '2014-01-28', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 0)
(6, 64, 1, 1, '2014-02-17 15:41:08', '2014-03-18 18:04

### ... to be continued

In [ ]:
get_tables(db_path)

--------------